**Задача:** построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

**Цель:** построить модель для задачи классификации, которая выберет подходящий тариф

**Данные:**  нам предоставлены данные о поведении клиентов, которые уже перешли на эти тарифы. Предподготовку данных проводить не нужно, так как данные почищены, все возможные дубликаты удалены и пропуски обработаны (это проект ранних работ)

**План исследования:** построить минимум 3 модели и выяснить какая из них оптимальная для поставленной задачи.

In [1]:
# подключим библиотеки

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
# откроем файл с данными

try:
    data = pd.read_csv('users_behavior.csv')
except:
    data = pd.read_csv('/datasets/users_behavior.csv')
data  

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [3]:
# проверим что данные чистые

data.describe(include='all')

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [4]:
# Разобьем данные на две выборки (обучающую и тестовую 25%)
data_train_total, data_test = train_test_split(data, test_size=0.25, random_state=12345)

# Разобьем обучающие данные на две выборки (обучающую и валидационную 25% от общего количества)
data_train, data_valid = train_test_split(data_train_total, test_size=0.3334, random_state=12345)

# проверим, верно ли разбились данные
print('Обучающаяся выборка', len(data_train))
print('Валидационная выборка', len(data_valid))
print('Тестовая выборка', len(data_test))

Обучающаяся выборка 1606
Валидационная выборка 804
Тестовая выборка 804


In [5]:
# подготовим данные для моделей

features_train = data_train.drop('is_ultra', axis=1)
target_train = data_train['is_ultra']
features_valid = data_valid.drop('is_ultra', axis=1)
target_valid = data_valid['is_ultra']
features_test = data_test.drop('is_ultra', axis=1)
target_test = data_test['is_ultra']

In [7]:
# проверим вариант модели на обучающей выборке- Дерево решений
# зададим гиперпараметры, которые позволят ограничит обучение модели

best_model = None
best_result = 0

for depth in range(1, 5):
    model = DecisionTreeClassifier(random_state=12345, criterion='gini', min_samples_split=2, min_samples_leaf=1, max_depth = depth) # обучим модель с заданной глубиной дерева	
    model.fit(features_train, target_train) # обучим модель	
    predictions_valid = model.predict(features_valid) # получим предсказания модели	
    result = accuracy_score(target_valid, predictions_valid) # посчитаем качество модели	
    if result > best_result:
        best_model = model
        best_result = result

print(f'Accuracy лучшей модели Дерево решений: {best_result}, глубина - {depth}') 


Accuracy лучшей модели Дерево решений: 0.7873134328358209, глубина - 4


In [8]:
# проверим вариант модели - Случайный лес

best_model = None
best_result = 0
for est in range(1, 10):
    model = RandomForestClassifier(random_state=12345, criterion='gini', min_samples_split=2, min_samples_leaf=1, n_estimators = est) # обучим модель с заданной кол-ом деревьев	
    model.fit(features_train, target_train) # обучим модель	
    predictions_valid = model.predict(features_valid) # получим предсказания модели	
    result = model.score(features_valid, target_valid) # посчитаем качество модели	
    if result > best_result:	
        best_model = model
        best_result = result
     
print(f'Accuracy лучшей модели Случайный лес: {best_result}, кол-во деревьев - {est}')


Accuracy лучшей модели Случайный лес: 0.8097014925373134, кол-во деревьев - 9


In [9]:
# проверим вариант модели - Логическая регрессия

model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000) 
model.fit(features_train, target_train) 
model.predict(features_valid) 
model.score(features_valid, target_valid) 

print("Accuracy лучшей модели Логическая регрессия:", model.score(features_valid, target_valid))


Accuracy лучшей модели Логическая регрессия: 0.7375621890547264


Самый высокая метрика точности у валидативной выборки в модели случайный лес - 80% верных ответов

## Проверим модель на тестовой выборке

In [10]:
model_test = RandomForestClassifier(random_state=12345, n_estimators = 9) # зададим модель с заданным кол-ом деревьев	
model_test.fit(features_train, target_train) # обучим модель	
predictions_test = model_test.predict(features_test) # получим предсказания модели по тестовой выборке
result_test = accuracy_score(predictions_test, target_test) # посчитаем качество модели	тестовой выборки
    
print("Accuracy тестовой выборки модели Случайный лес:", result_test)

Accuracy тестовой выборки модели Случайный лес: 0.7786069651741293


Точность ответов тестовой выборки так же велика - 77% верных ответов

## Проверим модели на адекватность

In [11]:
from sklearn.dummy import DummyClassifier
dummy_adek = DummyClassifier(strategy="most_frequent", random_state=0)
dummy_adek.fit(features_train, target_train)
dummy_adek.score(features_train, target_train)

0.6973848069738481

Обычная модель ошибается в 30% данных, выбранная модель "Случайный лес" ошибается в 20% данных и превышает нижний порок по заданию 75%. Из чего делаем вывод, что выбранняа модель адекватна.

ВЫВОД:
Для построения модели, определящей оптимальный тариф, построим минимум 3 модели и выясним какая из них оптимальная для поставленной задачи.
Анализ проводился на данных о поведении клиентов, которые уже перешли на тарифы «Смарт» или «Ультра». Предподготовку данных проводить не нужно, так как данные почищены, все возможные дубликаты удалены и пропуски обработаны.
При обработке данных использовались 3 модели DecisionTreeClassifier, RandomForestClassifier, LogisticRegression. Самую высокую точность дала модель Случайный лес (RandomForestClassifier), ее метрика составляет 80% верных ответов, в то время как обычная модель давала бы только 70% верных ответов.